In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

In [3]:
dataset = pd.read_csv("https://full-stack-bigdata-datasets.s3.eu-west-3.amazonaws.com/Machine+Learning+Supervis%C3%A9/R%C3%A9gression+r%C3%A9gularis%C3%A9es/gene+data/data_lasso.csv",)
dataset.head()

Unnamed: 0.1 Unnamed: 0  gene_0    gene_1    gene_2    gene_3     gene_4  \
0             0   sample_0     0.0  2.017209  3.265527  5.478487  10.431999   
1             1   sample_1     0.0  0.592732  1.588421  7.586157   9.623011   
2             2   sample_2     0.0  3.511759  4.327199  6.881787   9.870730   
3             3   sample_3     0.0  3.663618  4.507649  6.659068  10.196184   
4             4   sample_4     0.0  2.655741  2.821547  6.539454   9.738265   

   gene_5    gene_6    gene_7  ...  gene_20522  gene_20523  gene_20524  \
0     0.0  7.175175  0.591871  ...    8.210257    9.723516    7.220030   
1     0.0  6.816049  0.000000  ...    7.323865    9.740931    6.256586   
2     0.0  6.972130  0.452595  ...    8.127123   10.908640    5.401607   
3     0.0  7.843375  0.434882  ...    8.792959   10.141520    8.942805   
4     0.0  6.566967  0.360982  ...    8.891425   10.373790    7.181162   

   gene_20525  gene_20526  gene_20527  gene_20528  gene_20529  gene_20530  \
0    9.119813   12.003135    9.650743    8.921326    5.286759         0.0   
1    8.381612   12.674552   10.517059    9.397854    2.094168         0.0   
2    9.911597    9.045255    9.788359   10.090470    1.683023         0.0   
3    9.601208   11.392682    9.694814    9.684365    3.292001         0.0   
4    9.846910   11.922439    9.217749    9.461191    5.110372         0.0   

      target  
0  12.408154  
1  13.414970  
2  13.566183  
3  12.943886  
4  91.307146  

[5 rows x 20534 columns]

The two first columns are useless, find a way to get rid of them.

In [6]:
# Sélection de toutes les features sauf les deux premières colonnes
dataset_filtered = dataset.iloc[:,2:]
dataset_filtered.head()

gene_0    gene_1    gene_2    gene_3     gene_4  gene_5    gene_6  \
0     0.0  2.017209  3.265527  5.478487  10.431999     0.0  7.175175   
1     0.0  0.592732  1.588421  7.586157   9.623011     0.0  6.816049   
2     0.0  3.511759  4.327199  6.881787   9.870730     0.0  6.972130   
3     0.0  3.663618  4.507649  6.659068  10.196184     0.0  7.843375   
4     0.0  2.655741  2.821547  6.539454   9.738265     0.0  6.566967   

     gene_7  gene_8  gene_9  ...  gene_20522  gene_20523  gene_20524  \
0  0.591871     0.0     0.0  ...    8.210257    9.723516    7.220030   
1  0.000000     0.0     0.0  ...    7.323865    9.740931    6.256586   
2  0.452595     0.0     0.0  ...    8.127123   10.908640    5.401607   
3  0.434882     0.0     0.0  ...    8.792959   10.141520    8.942805   
4  0.360982     0.0     0.0  ...    8.891425   10.373790    7.181162   

   gene_20525  gene_20526  gene_20527  gene_20528  gene_20529  gene_20530  \
0    9.119813   12.003135    9.650743    8.921326    5.286759         0.0   
1    8.381612   12.674552   10.517059    9.397854    2.094168         0.0   
2    9.911597    9.045255    9.788359   10.090470    1.683023         0.0   
3    9.601208   11.392682    9.694814    9.684365    3.292001         0.0   
4    9.846910   11.922439    9.217749    9.461191    5.110372         0.0   

      target  
0  12.408154  
1  13.414970  
2  13.566183  
3  12.943886  
4  91.307146  

[5 rows x 20532 columns]

# Exploration
Print some information about the dataset. Do you notice something unusual ?


In [8]:
# Affichage du nombre de lignes dans le dataset
print(f"Number of rows: {dataset_filtered.shape[0]}\n")
print(f"Number of columns: {dataset_filtered.shape[1]}\n")

# Affichage des premières lignes du dataset filtré
print("Display of dataset (first 5 rows):")
display(dataset_filtered.head())
print()

# Statistiques descriptives de base pour le dataset
print("Basic statistics: ")
data_desc = dataset_filtered.describe(include='all')
display(data_desc)
print()

# Calcul et affichage du pourcentage de valeurs manquantes par colonne
print("Percentage of missing values per column: ")
missing_values = (100 * dataset_filtered.isnull().sum() / dataset_filtered.shape[0]).round(2)
display(missing_values)

Number of rows: 801

Number of columns: 20532

Display of dataset (first 5 rows):


gene_0    gene_1    gene_2    gene_3     gene_4  gene_5    gene_6  \
0     0.0  2.017209  3.265527  5.478487  10.431999     0.0  7.175175   
1     0.0  0.592732  1.588421  7.586157   9.623011     0.0  6.816049   
2     0.0  3.511759  4.327199  6.881787   9.870730     0.0  6.972130   
3     0.0  3.663618  4.507649  6.659068  10.196184     0.0  7.843375   
4     0.0  2.655741  2.821547  6.539454   9.738265     0.0  6.566967   

     gene_7  gene_8  gene_9  ...  gene_20522  gene_20523  gene_20524  \
0  0.591871     0.0     0.0  ...    8.210257    9.723516    7.220030   
1  0.000000     0.0     0.0  ...    7.323865    9.740931    6.256586   
2  0.452595     0.0     0.0  ...    8.127123   10.908640    5.401607   
3  0.434882     0.0     0.0  ...    8.792959   10.141520    8.942805   
4  0.360982     0.0     0.0  ...    8.891425   10.373790    7.181162   

   gene_20525  gene_20526  gene_20527  gene_20528  gene_20529  gene_20530  \
0    9.119813   12.003135    9.650743    8.921326    5.286759         0.0   
1    8.381612   12.674552   10.517059    9.397854    2.094168         0.0   
2    9.911597    9.045255    9.788359   10.090470    1.683023         0.0   
3    9.601208   11.392682    9.694814    9.684365    3.292001         0.0   
4    9.846910   11.922439    9.217749    9.461191    5.110372         0.0   

      target  
0  12.408154  
1  13.414970  
2  13.566183  
3  12.943886  
4  91.307146  

[5 rows x 20532 columns]


Basic statistics: 


gene_0      gene_1      gene_2      gene_3      gene_4  gene_5  \
count  801.000000  801.000000  801.000000  801.000000  801.000000   801.0   
mean     0.026642    3.010909    3.095350    6.722305    9.813612     0.0   
std      0.136850    1.200828    1.065601    0.638819    0.506537     0.0   
min      0.000000    0.000000    0.000000    5.009284    8.435999     0.0   
25%      0.000000    2.299039    2.390365    6.303346    9.464466     0.0   
50%      0.000000    3.143687    3.127006    6.655893    9.791599     0.0   
75%      0.000000    3.883484    3.802534    7.038447   10.142324     0.0   
max      1.482332    6.237034    6.063484   10.129528   11.355621     0.0   

           gene_6      gene_7      gene_8      gene_9  ...  gene_20522  \
count  801.000000  801.000000  801.000000  801.000000  ...  801.000000   
mean     7.405509    0.499882    0.016744    0.013428  ...    8.765891   
std      1.108237    0.508799    0.133635    0.204722  ...    0.603176   
min      3.930747    0.000000    0.000000    0.000000  ...    6.678368   
25%      6.676042    0.000000    0.000000    0.000000  ...    8.383834   
50%      7.450114    0.443076    0.000000    0.000000  ...    8.784144   
75%      8.121984    0.789354    0.000000    0.000000  ...    9.147136   
max     10.718190    2.779008    1.785592    4.067604  ...   11.105431   

       gene_20523  gene_20524  gene_20525  gene_20526  gene_20527  gene_20528  \
count  801.000000  801.000000  801.000000  801.000000  801.000000  801.000000   
mean    10.056252    4.847727    9.741987   11.742228   10.155271    9.590726   
std      0.379278    2.382728    0.533898    0.670371    0.580569    0.563849   
min      8.669456    0.000000    7.974942    9.045255    7.530141    7.864533   
25%      9.826027    3.130750    9.400747   11.315857    9.836525    9.244219   
50%     10.066385    5.444935    9.784524   11.749802   10.191207    9.566511   
75%     10.299025    6.637412   10.082269   12.177852   10.578561    9.917888   
max     11.318243    9.207495   11.811632   13.715361   11.675653   12.813320   

       gene_20529  gene_20530      target  
count  801.000000  801.000000  801.000000  
mean     5.528177    0.095411   41.219409  
std      2.073859    0.364529   37.655049  
min      0.593975    0.000000    0.000000  
25%      4.092385    0.000000   11.433002  
50%      5.218618    0.000000   13.415959  
75%      6.876382    0.000000   89.166837  
max     11.205836    5.254133  100.000000  

[8 rows x 20532 columns]


Percentage of missing values per column: 


gene_0        0.0
gene_1        0.0
gene_2        0.0
gene_3        0.0
gene_4        0.0
             ... 
gene_20527    0.0
gene_20528    0.0
gene_20529    0.0
gene_20530    0.0
target        0.0
Length: 20532, dtype: float64

There are more than 20k features in this dataset 🤯 Actually, it's very common in genetics.

Make sure there are no missing values at all in the whole dataset

In [9]:
if not dataset_filtered.isnull().any().any():
    print("No missing values found in the dataset.")
else:
    print("There are missing values in the dataset.")

No missing values found in the dataset.


# Preparing the data for machine learning
Separate the target from the features

In [10]:
# Séparation de la variable cible et des caractéristiques
print("Separating labels from features...")
target_variable = "target"

# Vérification de l'existence de la variable cible dans le jeu de données
if target_variable in dataset_filtered.columns:
    X = dataset_filtered.drop(columns=[target_variable])
    y = dataset_filtered[target_variable]
    print("...Done.\n")

    print("y (Target Variable):")
    print(y.head(), "\n")
    
    print("X (Features):")
    print(X.head())
else:
    print(f"Error: The target variable '{target_variable}' does not exist in the dataset.")

Separating labels from features...
...Done.

y (Target Variable):
0    12.408154
1    13.414970
2    13.566183
3    12.943886
4    91.307146
Name: target, dtype: float64 

X (Features):
   gene_0    gene_1    gene_2    gene_3     gene_4  gene_5    gene_6  \
0     0.0  2.017209  3.265527  5.478487  10.431999     0.0  7.175175   
1     0.0  0.592732  1.588421  7.586157   9.623011     0.0  6.816049   
2     0.0  3.511759  4.327199  6.881787   9.870730     0.0  6.972130   
3     0.0  3.663618  4.507649  6.659068  10.196184     0.0  7.843375   
4     0.0  2.655741  2.821547  6.539454   9.738265     0.0  6.566967   

     gene_7  gene_8  gene_9  ...  gene_20521  gene_20522  gene_20523  \
0  0.591871     0.0     0.0  ...    4.926711    8.210257    9.723516   
1  0.000000     0.0     0.0  ...    4.593372    7.323865    9.740931   
2  0.452595     0.0     0.0  ...    5.125213    8.127123   10.908640   
3  0.434882     0.0     0.0  ...    6.076566    8.792959   10.141520   
4  0.360982     0.0  

Make a train/test splitting

In [11]:
# Division du dataset en ensembles d'entraînement et de test
print("Dividing dataset into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(f"Train set: {X_train.shape[0]} samples, Test set: {X_test.shape[0]} samples")
print("...Done.\n")

Dividing dataset into train and test sets...
Train set: 640 samples, Test set: 161 samples
...Done.



# Preprocessing
What preprocessings are necessary here ? Apply them to the features

In [12]:
print(dataset_filtered.dtypes)

gene_0        float64
gene_1        float64
gene_2        float64
gene_3        float64
gene_4        float64
               ...   
gene_20527    float64
gene_20528    float64
gene_20529    float64
gene_20530    float64
target        float64
Length: 20532, dtype: object


In [13]:
# Détection automatique des colonnes numériques et catégorielles
numeric_features = [col for col in X.columns if X[col].dtype in ['float64', 'int64']]
categorical_features = [col for col in X.columns if X[col].dtype == 'object']

print("Found numeric features:", numeric_features)
print("Found categorical features:", categorical_features)

Found numeric features: ['gene_0', 'gene_1', 'gene_2', 'gene_3', 'gene_4', 'gene_5', 'gene_6', 'gene_7', 'gene_8', 'gene_9', 'gene_10', 'gene_11', 'gene_12', 'gene_13', 'gene_14', 'gene_15', 'gene_16', 'gene_17', 'gene_18', 'gene_19', 'gene_20', 'gene_21', 'gene_22', 'gene_23', 'gene_24', 'gene_25', 'gene_26', 'gene_27', 'gene_28', 'gene_29', 'gene_30', 'gene_31', 'gene_32', 'gene_33', 'gene_34', 'gene_35', 'gene_36', 'gene_37', 'gene_38', 'gene_39', 'gene_40', 'gene_41', 'gene_42', 'gene_43', 'gene_44', 'gene_45', 'gene_46', 'gene_47', 'gene_48', 'gene_49', 'gene_50', 'gene_51', 'gene_52', 'gene_53', 'gene_54', 'gene_55', 'gene_56', 'gene_57', 'gene_58', 'gene_59', 'gene_60', 'gene_61', 'gene_62', 'gene_63', 'gene_64', 'gene_65', 'gene_66', 'gene_67', 'gene_68', 'gene_69', 'gene_70', 'gene_71', 'gene_72', 'gene_73', 'gene_74', 'gene_75', 'gene_76', 'gene_77', 'gene_78', 'gene_79', 'gene_80', 'gene_81', 'gene_82', 'gene_83', 'gene_84', 'gene_85', 'gene_86', 'gene_87', 'gene_88', 'gene_

In [14]:
# Pipeline de transformation pour les caractéristiques numériques
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Remplace les valeurs manquantes par la moyenne des colonnes
    ('scaler', StandardScaler())  # Met à l'échelle les données en les centrant et les réduisant
])

print("Numeric feature transformer pipeline created.")

Numeric feature transformer pipeline created.


In [15]:
# Création de l'objet de prétraitement qui applique les transformations aux colonnes numériques
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),     # Applique le pipeline numérique aux caractéristiques numériques
    ]
)

print("Preprocessor object created with transformations for numeric features.")

Preprocessor object created with transformations for numeric features.


In [16]:
# Prétraitement sur l'ensemble d'entraînement
print("Performing preprocessing on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print("...Done.")

# Convertit en DataFrame pour garder les noms de colonnes après transformation
X_train = pd.DataFrame(X_train, columns=preprocessor.get_feature_names_out())
print(X_train.head())

print()

# Prétraitement sur l'ensemble de test
print("Performing preprocessing on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test)  # Ne pas refit ici
print("...Done.")

# Convertit en DataFrame pour garder les noms de colonnes après transformation
X_test = pd.DataFrame(X_test, columns=preprocessor.get_feature_names_out())
print(X_test.head())
print()

Performing preprocessing on train set...
      gene_0    gene_1    gene_2    gene_3     gene_4  gene_5    gene_6  \
364  0.00000  0.000000  2.275752  6.025051  10.181674     0.0  6.112777   
458  0.75745  1.915330  1.182565  6.282078   8.972058     0.0  5.440902   
76   0.00000  3.195537  2.275156  5.719778   9.145591     0.0  7.333683   
64   0.00000  2.949311  4.597877  7.095460   9.762425     0.0  6.493412   
638  0.00000  3.777693  3.417920  7.031164   9.960924     0.0  8.119844   

       gene_7  gene_8  gene_9  ...  gene_20521  gene_20522  gene_20523  \
364  0.565890     0.0     0.0  ...    5.402804    7.875964   10.120121   
458  0.000000     0.0     0.0  ...    7.455081    8.825525   10.021993   
76   1.192762     0.0     0.0  ...    6.534924    9.382676    9.893725   
64   0.000000     0.0     0.0  ...    6.066335    9.048356   10.287493   
638  0.605020     0.0     0.0  ...    6.012622    9.200089   10.349337   

     gene_20524  gene_20525  gene_20526  gene_20527  gene_20528

# Ridge

Let's focus on Ridge regularization. We'll train 3 Ridge regressors with different values of the strength α, and analyze the performances as well as the influence on the model's coefficients.

Declare an instance of the Ridge class with α =1. Save this instance for later analysis into an object called ridge1. Train the model and display its R2-score on train and test sets.

In [21]:
# 3-fold cross-validation avec modèle Ridge pour évaluer le score R2
print("Performing 3-fold cross-validation with Ridge regression...")
ridge1 = Ridge()
scores = cross_val_score(ridge1, X_train, y_train, cv=3, scoring='r2')

print(f'The mean cross-validated R2-score is: {scores.mean():.4f}')
print(f'The standard deviation of R2-scores is: {scores.std():.4f}')

Performing 3-fold cross-validation with Ridge regression...
The mean cross-validated R2-score is: 0.9808
The standard deviation of R2-scores is: 0.0027


In [22]:
# Recherche par grille (Grid Search) pour la régression Ridge
print("Performing grid search with cross-validation for Ridge regression...")

# Grille des valeurs de l'hyperparamètre alpha à tester
params = {
    'alpha': [1.0]
}

# Configuration de la recherche par grille avec validation croisée
gridsearch = GridSearchCV(estimator=ridge1, param_grid=params, cv=3, scoring='r2', n_jobs=-1, verbose=1)
gridsearch.fit(X_train, y_train)

print("...Done.")
print("Best hyperparameters:", gridsearch.best_params_)
print(f"Best cross-validated R2 score: {gridsearch.best_score_:.4f}")

# Print R^2 scores
print("R2 score on training set : ", gridsearch.score(X_train, y_train))
print("R2 score on test set : ", gridsearch.score(X_test, y_test))

Performing grid search with cross-validation for Ridge regression...
Fitting 3 folds for each of 1 candidates, totalling 3 fits
...Done.
Best hyperparameters: {'alpha': 1.0}
Best cross-validated R2 score: 0.9808
R2 score on training set :  0.9999999996538701
R2 score on test set :  0.9867318246730055


Declare an instance of the Ridge class with α = 1000000. Save this instance for later analysis into an object called ridge2. Train the model and display its R2-score on train and test sets.

In [23]:
# 3-fold cross-validation avec modèle Ridge pour évaluer le score R2
print("Performing 3-fold cross-validation with Ridge regression...")
ridge2 = Ridge()
scores = cross_val_score(ridge2, X_train, y_train, cv=3, scoring='r2')

print(f'The mean cross-validated R2-score is: {scores.mean():.4f}')
print(f'The standard deviation of R2-scores is: {scores.std():.4f}')

Performing 3-fold cross-validation with Ridge regression...
The mean cross-validated R2-score is: 0.9808
The standard deviation of R2-scores is: 0.0027


In [ ]:
# Recherche par grille (Grid Search) pour la régression Ridge
print("Performing grid search with cross-validation for Ridge regression...")

# Grille des valeurs de l'hyperparamètre alpha à tester
params = {
    'alpha': [1000000]
}

# Configuration de la recherche par grille avec validation croisée
gridsearch = GridSearchCV(estimator=ridge1, param_grid=params, cv=3, scoring='r2', n_jobs=-1, verbose=1)
gridsearch.fit(X_train, y_train)

print("...Done.")
print("Best hyperparameters:", gridsearch.best_params_)
print(f"Best cross-validated R2 score: {gridsearch.best_score_:.4f}")

# Print R^2 scores
print("R2 score on training set : ", gridsearch.score(X_train, y_train))
print("R2 score on test set : ", gridsearch.score(X_test, y_test))